# Chapter 9: Conditional Expectation
 
This Jupyter notebook is the Python equivalent of the R code in section 9.8 R, pp. 408 - 410, [Introduction to Probability, 1st Edition](https://www.crcpress.com/Introduction-to-Probability/Blitzstein-Hwang/p/book/9781466575578), Blitzstein & Hwang.

----

## Mystery prize simulation

We can use simulation to show that in Example 9.1.7, the example of bidding on a mystery prize with unknown value, any bid will lead to a negative payout on average. First choose a bid `b` (we chose 0.6); then simulate a large number of hypothetical mystery prizes and store them in `v`:

The bid is accepted if `b > (2/3)*v`. To get the average profit conditional on an accepted bid, we use square brackets to keep only those values of `v` satisfying the condition:

This value is negative regardless of `b`, as you can check by experimenting with different values of`b`.

## Time until HH vs. HT

To verify the results of Example 9.1.9, we can start by generating a long sequence of fair coin tosses. This is done with the `sample` command. We use `paste` with the `collapse=""` argument to turn these tosses into a single string of `H`'s and `T`'s:

A sequence of length 100 is enough to virtually guarantee that both `HH` and `HT` will have appeared at least once.

To determine how many tosses are required on average to see `HH` and `HT`, we need to generate many sequences of coin tosses. For this, we use our familiar friend `replicate`:

Now `r` contains a thousand sequences of coin tosses, each of length 100. To find the first appearance of `HH` in each of these sequences, it is easiest to use the `str_locate` command, located in the `stringr` package. After you've installed and loaded the package,

creates a two-column table `t`, whose columns contain the starting and ending positions of the first appearance of `HH` in each sequence of coin tosses. (Use `head(t)` to display the first few rows of the table and get an idea of what your results look like.) What we want are the ending positions, given by the second column. In particular, we want the average value of the second column, which is an approximation of the average waiting time for `HH`:

Is your answer around 6? Trying again with `HT` instead of `HH`, is your answer around 4?

## Linear regression

In Example 9.3.10, we derived formulas for the slope and intercept of a linear regression model, which can be used to predict a response variable using an explanatory variable. Let's try to apply these formulas to a simulated dataset:

Here cov(x,y), var(x), and mean(x) provide the sample covariance, sample variance, and sample mean, estimating the quantities Cov(X, Y), Var(X), and E(X), respectively. (We have discussed sample mean and sample variance in detail in earlier chapters. Sample covariance is defined analogously, and is a natural way to estimate the true covariance.)

You should find that b is close to 5 and a is close to 3. These estimated values define the line of best fit. The abline command lets us plot the _line of best fit_ on top of our scatterplot:

The first argument to abline is the intercept of the line, and the second argument is the slope.

----

&copy; Blitzstein, Joseph K.; Hwang, Jessica. Introduction to Probability (Chapman & Hall/CRC Texts in Statistical Science).